# RAG: AgroDoctor

## Notebook content

This notebook contains the steps and code to demonstrate the retrieval-augmented generation pattern in IBM watsonx.ai on IBM Cloud. If you are using IBM watsonx.ai on-premises, then see the [other versions of this notebook](#cpd-only).

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to apply the retrieval-augmented generation pattern to a question-answering use case in watsonx.ai.


## Scenario
The website for an online seed catalog has many articles to help customers plan their garden and ultimately select which seeds to purchase. A new widget is being added to the website to answer customer questions based on the contents of the articles.


## Contents

This notebook contains the following parts:

- [Overview of retrieval-augmented generation](#overview)
- [Step 1: Set up prerequisites](#setup)
- [Step 2: Create a knowledge base](#knowledgebase)
- [Step 3: Build a simple search component](#search)
- [Step 4: Craft prompt text](#prompt)
- [Step 5: Generate output using the foundation models Python library](#generate)
- [Step 6: Pull everything together to perform retrieval-augmented generation](#rag)
- [Summary](#summary)

<a id="overview"></a>
## Overview of retrieval-augmented generation

The retrieval-augmented generation pattern involves three basic steps:
1. Search for relevant content in your knowledge base
2. Pull the most relevant content into your prompt as context
3. Send the combined prompt text to a foundation model to generate output

The term _retrieval-augmented generation_ (RAG) was introduced in this paper: <a href="https://arxiv.org/abs/2005.11401" target="_blank" rel="noopener no referrer">Retrieval-augmented generation for knowledge-intensive NLP tasks</a>

> "We build RAG models where the parametric memory is a pre-trained seq2seq transformer, and the
non-parametric memory is a dense vector index of Wikipedia, accessed with a pre-trained neural
retriever."

In that paper, the term "RAG models" refers to a specific implementation of a _retriever_ (a specific query encoder and vector-based document search index) and a _generator_ (a specific pre-trained, generative language model.) 

However, the basic search-and-generate approach can be generalized to use different retriever components and foundation models.

In this notebook:
- The **knowledge base** is a list of two articles
- The **retrieval component** consists of a simple search function
- The **generate** component uses the foundation model Python library in watsonx.ai


<a id="setup"></a>
# Step 1: Set up prerequisites

Before you use the sample code in this notebook, you must perform setup tasks.

## 1.1 Associate an instance of the Watson Machine Learning service with the current project

The _current project_ is the project in which you are running this notebook.

If an instance of Watson Machine Learning is not already associated with the current project, follow the instructions in this topic to do so: [Adding associated services to a project](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?context=wx&audience=wdp)


## 1.2 Create an IBM Cloud API key

Create an IBM Cloud API key by following these instructions: [Creating an IBM Cloud API key](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key)

Then paste your new IBM Cloud API key in the code cell below.

In [76]:
cloud_apikey = "api-key"

## 1.3 Define a credentials object with the IBM Cloud API key

In [77]:
credentials = { 
    "url"    : "https://us-south.ml.cloud.ibm.com", 
    "apikey" : cloud_apikey
}

<a id="knowledgebase"></a>
# Step 2: Create a knowledge base

In this notebook, the knowledge base is a collection of two articles.  

(These articles were written as samples for watsonx.ai, they are not real articles published anywhere else.  The authors and publication dates are fictional.)

In [78]:
article_01 = \
"Banana plants are susceptible to several diseases, including Black Sigatoka, Panama Disease,"\
"Moko Disease, Banana Bunchy Top Virus (BBTV), and Yellow Sigatoka. Black Sigatoka, caused by"\
"the Pseudocercospora fijiensis fungus, manifests as black streaks and spots on leaves, leading"\
"to leaf necrosis. Panama Disease, a soil-borne infection caused by Fusarium oxysporum, results"\
"in yellowing and wilting of leaves. Moko Disease, caused by Ralstonia solanacearum, leads to"\
"leaf yellowing and blackened fruits, while BBTV, a viral disease, causes stunted, bunched leaves"\
"with dark green streaks. Yellow Sigatoka produces yellow spots that turn necrotic, impacting"\
"leaf function. Remedies include applying fungicides like mancozeb for fungal infections, improving"\
"air circulation by pruning plants, using disease-resistant varieties,"\
"and following sanitation practices to prevent the spread of bacteria and viruses."


In [79]:
article_02 = \
"Jackfruit plants are vulnerable to diseases such as Pink Disease, Stem Rot, Anthracnose,"\
"and Bacterial Leaf Spot. Pink Disease, caused by the fungus Corticium salmonicolor,"\
"leads to pink-colored patches on stems and branches, eventually causing dieback."\
"Stem Rot is caused by Pythium fungi, resulting in soft, water-soaked lesions at the base of the plant,"\
"leading to stem collapse. Anthracnose, caused by Colletotrichum species, produces dark, sunken spots on leaves,"\
"stems, and fruits, leading to defoliation and fruit rot. Bacterial Leaf Spot, caused by Xanthomonas campestris,"\
"presents as water-soaked lesions that turn brown and necrotic, affecting leaf function. Remedies include"\
"pruning infected parts, applying copper-based fungicides for fungal infections, improving drainage to prevent"\
"root rot, and practicing proper sanitation to reduce bacterial spread."

In [80]:
article_03=\
"Rice plants are susceptible to several leaf diseases that can severely impact their growth and productivity."\
"Among the most common are Bacterial Leaf Blight, Brown Spot, and Leaf Smut. Bacterial Leaf Blight,"\
"caused by the bacterium Xanthomonas oryzae pv. oryzae, initially appears as water-soaked streaks along the edges of the leaves."\
"These streaks turn yellowish-brown and become necrotic, eventually leading to leaf wilting and drying, which restricts photosynthesis and stunts the plant's growth."\
"To control Bacterial Leaf Blight, farmers should use resistant rice varieties, practice crop rotation, apply copper-based bactericides, manage water effectively,"\
"and avoid excessive nitrogen fertilizer.Brown Spot, caused by the fungus Bipolaris oryzae, manifests as brown spots with gray centers on the leaves,"\
"and severe infections give fields a scorched appearance. This disease affects grain quality and reduces yield, especially in nutrient-deficient or"\
"drought-stressed plants. To manage Brown Spot, using resistant varieties, applying balanced fertilizers (especially potassium),"\
"and treating fields with fungicides like mancozeb can be effective, along with ensuring good drainage and reducing plant stress."\
"Leaf Smut, caused by the fungal pathogen Entyloma oryzae, causes small black spots or streaks on rice leaves, particularly in older plants."\
"While the damage to photosynthesis is minimal, severe infections can lead to premature leaf death and reduced plant vigor."\
"Controlling Leaf Smut involves using disease-free seeds, crop rotation, and foliar fungicides such as carbendazim or thiophanate-methyl."\
"Proper sanitation, including the removal of infected plant debris, is also critical in reducing the spread of this disease."\
"Effective management of these diseases involves a combination of resistant varieties, timely application of fungicides or bactericides,"\
"and good agronomic practices to minimize losses in rice production."


In [81]:
article_04=\
"Tea plants are vulnerable to a range of diseases and pests that can affect leaf health and yield. The most common tea leaf diseases include Black Rot,"\
"Brown Blight, Leaf Rust, White Spot, as well as infestations by Red Spider Mites and Tea Mosquito Bugs.Black Rot of Tea, caused by the fungus Corticium invisum,"\
"manifests as black, water-soaked lesions on leaves, leading to defoliation and reduced plant vigor. The fungus thrives in warm and humid conditions."\
"Effective control measures include pruning infected parts, improving air circulation, and applying copper-based fungicides.Brown Blight,"\
"caused by the fungus Colletotrichum camelliae, appears as brown spots on tea leaves that gradually expand,leading to leaf drop and reduced productivity. "\
"The disease is most prevalent during wet weather. To manage Brown Blight, it is recommended to prune infected areas,"\
"maintain proper plant spacing to enhance airflow, and apply fungicides such as mancozeb or chlorothalonil.Leaf Rust, caused by Puccinia theae,"\
"results in rusty orange pustules on the underside of leaves, leading to premature defoliation. This disease weakens the tea plants and impacts crop yield. "\
"Management strategies include using resistant varieties, applying sulfur-based fungicides, and avoiding excessive irrigation.White Spot of Tea, "\
"caused by the fungus Pestalotiopsis theae, produces small white lesions on tea leaves, eventually leading to leaf curling and browning. "\
"The spread of this disease can be minimized by regular pruning, sanitation practices to remove infected leaves, and the application of fungicides"\
"like thiophanate-methyl.Tea plants are also infested by Red Spider Mites, tiny arachnids that feed on leaf sap, causing stippling, yellowing, and"\
"premature leaf drop. Infestations often occur during dry weather. Effective management includes the use of acaricides, maintaining humidity levels"\
"to discourage mites, and encouraging natural predators like ladybugs.Tea Mosquito Bugs, (Helopeltis theivora), are another significant pest,"\
"causing black necrotic spots on tea leaves and young shoots. Severe infestations lead to stunted growth and reduced yield. Control methods involve " \
"using insecticides, such as neem oil or synthetic pyrethroids, and promoting biological control agents like parasitic wasps.By using resistant cultivars,"\
"applying appropriate fungicides and insecticides, and maintaining proper cultivation practices like pruning and ensuring good sanitation,"\
"farmers can effectively manage these diseases and pest infestations, protecting tea leaf health and improving yield."

In [82]:
article_05= \
"Coffee plants are susceptible to several diseases and pests that can severely impact leaf health and productivity."\
"Some of the most common issues affecting coffee leaves include Brown Eye Spot, Leaf Miner, Leaf Rust, and Red Spider Mite infestations."\
"Brown Eye Spot, caused by the fungus Cercospora coffeicola, manifests as small brown lesions with a characteristic yellow halo,"\
"primarily affecting older leaves. The disease can reduce photosynthetic capacity and weaken the plant. Management includes improving air circulation,"\
"applying copper-based fungicides, and removing infected leaves to limit the spread.Leaf Miner, a pest caused by the larvae of the moth Leucoptera coffeella,"\
"burrows into the leaf tissue, creating winding tunnels that lead to necrosis and premature leaf drop. Infested leaves display pale,"\
"serpentine trails that can significantly reduce coffee yield. Control measures involve the use of biological agents like parasitoid wasps "\
"or the application of insecticides targeting the larvae.Leaf Rust, caused by the fungus Hemileia vastatrix, is a devastating disease for coffee plants. "\
"It appears as yellow-orange powdery lesions on the underside of leaves, leading to defoliation and weakened plants."\
"Leaf Rust thrives in warm and humid conditions.Effective control involves using resistant varieties, applying copper-based fungicides, "\
"and improving plant spacing for better air circulation.Red Spider Mites, tiny pests that feed on coffee leaf sap,cause stippling and yellowing,"\
"which can eventually lead to leaf drop.Infestations are more severe in dry conditions. Control strategies include increasing humidity around the plants,"\
"using acaricides,and encouraging natural predators such as predatory mites or ladybugs.By implementing integrated pest management (IPM) techniques,"\
"including fungicide and pesticide applications, regular pruning, sanitation, and the use of resistant coffee varieties,"\
"farmers can effectively mitigate these diseases and pests, promoting healthier coffee plants and improving crop yield."

In [83]:
article_06 = \
"Coconut plants are vulnerable to several diseases and pests that primarily affect their leaves, including Drying of Leaflets,"\
"Flaccidity, Infested Leaflets, Leaf Caterpillars, and Yellowing. These conditions can significantly reduce coconut yield if not "\
"properly managed. Drying of Leaflets is often caused by environmental stress, such as drought or poor irrigation. The leaflets turn brown and"\
"dry out, leading to reduced photosynthesis and weakening the plant. To manage this, ensure adequate watering, especially during dry seasons,"\
"and improve soil moisture retention through mulching.Flaccidity in coconut leaves is typically due to water stress or root damage,"\
"often associated with poor soil conditions or root diseases. The leaves appear wilted and lose their rigidity. Remedies include improving soil drainage,"\
"avoiding waterlogging, and providing consistent irrigation to maintain healthy root function.Infested Leaflets are caused by pests such as scale insects,"\
"mealybugs, or mites, which feed on the sap of the coconut leaves, causing them to become deformed and discolored. In severe cases,"\
"leaves may die and drop prematurely. Control strategies include applying horticultural oils or insecticides and introducing natural"\
"predators like ladybugs to control pest populations.Leaf Caterpillars, specifically the Opisina arenosella species, feed on the coconut leaflets,"\
"creating large, irregular holes that reduce the plant's photosynthetic capacity. Severe infestations can cause complete defoliation."\
"Effective control involves using biological agents such as parasitoid wasps, applying neem-based insecticides, and regularly removing and destroying"\
"infested leaves.Yellowing of coconut leaves is often linked to nutrient deficiencies, particularly nitrogen or potassium."\
"It can also be caused by diseases like Lethal Yellowing, a phytoplasma disease spread by leafhoppers. To address this, apply appropriate fertilizers"\
"based on soil testing, and if Lethal Yellowing is suspected, remove and destroy infected trees to prevent further spread.By implementing proper"\
"irrigation, soil management, pest control, and regular monitoring, coconut farmers can minimize the impact of these diseases and pests,"\
"ensuring healthy plant growth and sustained coconut production."

In [84]:
knowledge_base = [ 
    { 
        "title"     : "Banana leaf disease", 
        "Author"    : "general summary based on commonly known facts about plant pathology and agricultural practices.",
        "txt"       : article_01 
    }, 
    {
        "title"     : "Jackfruit leaf disease",
        "Author"    : " general summary based on commonly known facts about plant pathology and agricultural practices.",
        "txt"       : article_02 
    },
    {
        "title"     : "Rice leaf disease",
        "Author"    : " general summary based on commonly known facts about plant pathology and agricultural practices.",
        "txt"       : article_03 
    },
    {
        "title"     : "Tea leaf disease",
        "Author"    : " general summary based on commonly known facts about plant pathology and agricultural practices.",
        "txt"       : article_04 
    },
    {
        "title"     : "Coffee leaf disease",
        "Author"    : " general summary based on commonly known facts about plant pathology and agricultural practices.",
        "txt"       : article_05
    },
    {
        "title"     : "Coconut leaf disease",
        "Author"    : " general summary based on commonly known facts about plant pathology and agricultural practices.",
        "txt"       : article_06 
    }
]

<a id="search"></a>
# Step 3: Build a simple search component

Many articles that discuss retrieval-augmented generation assume the retrieval component uses a vector database.  

However, to perform the general retrieval-augmented generation pattern, any search-and-retrieve method that can reliably return relevant content from the knowledge base will do.

In this notebook, the search component is a trivial search function that returns the index of one or the other of the two articles in the knowledge base, based on a simple regular expression match.

In [85]:
import re

def search( query_in, knowledge_base_in ):
    if re.match( r".*banana.*", query_in, re.IGNORECASE ):
        return 0
    elif re.match( r".*jackfruit.*", query_in, re.IGNORECASE ):
        return 1
    elif re.match( r".*rice.*", query_in, re.IGNORECASE ):
        return 2
    elif re.match( r".*tea.*", query_in, re.IGNORECASE ):
        return 3
    elif re.match( r".*cofee.*", query_in, re.IGNORECASE ):
        return 4
    elif re.match( r".*coconut.*", query_in, re.IGNORECASE ):
        return 5
    return -1

In [86]:
index = search( "My banana leaves are turning yellow. What should I do?", knowledge_base )

if index >= 0:
    print( "Index: " + str( index ) + "\nArticle: \"" + knowledge_base[index]["title"] + "\"" )
else:
    print( "No matching content was found" )

Index: 0
Article: "Banana leaf disease"


<a id="prompt"></a>
# Step 4: Craft prompt text

In this notebook, the task to be performed is a question-answering task.

There is no one, best prompt for any given task.  However, models that have been instruction-tuned, such as ibm/granite-13b-chat-v2,`bigscience/mt0-xxl-13b`, `google/flan-t5-xxl-11b`, or `google/flan-ul2-20b`, can generally perform this task with the sample prompt below.  Conservative decoding methods tend towards succinct answers.

In the prompt below, notice two string placeholders (marked with `%s`) that will be replaced at generation time:
- The first placeholder will be replaced with the text of the relevant article from the knowledge base
- The second placeholder will be replaced with the question to be answered

In [87]:
prompt_template = """
Article:
###
%s
###

Answer the following question using only information from the article. 
Answer in a complete sentence, with proper capitalization and punctuation. 
If there is no good answer in the article, say "I don't know".

Question: %s
Answer: 
"""

def augment( template_in, context_in, query_in ):
    return template_in % ( context_in,  query_in )


In [88]:
query = "My jackfruit leaves has black spot on it. What should I do?"

article_txt = knowledge_base[1]["txt"]

augmented_prompt = augment( prompt_template, article_txt, query )

print( augmented_prompt )


Article:
###
Jackfruit plants are vulnerable to diseases such as Pink Disease, Stem Rot, Anthracnose,and Bacterial Leaf Spot. Pink Disease, caused by the fungus Corticium salmonicolor,leads to pink-colored patches on stems and branches, eventually causing dieback.Stem Rot is caused by Pythium fungi, resulting in soft, water-soaked lesions at the base of the plant,leading to stem collapse. Anthracnose, caused by Colletotrichum species, produces dark, sunken spots on leaves,stems, and fruits, leading to defoliation and fruit rot. Bacterial Leaf Spot, caused by Xanthomonas campestris,presents as water-soaked lesions that turn brown and necrotic, affecting leaf function. Remedies includepruning infected parts, applying copper-based fungicides for fungal infections, improving drainage to preventroot rot, and practicing proper sanitation to reduce bacterial spread.
###

Answer the following question using only information from the article. 
Answer in a complete sentence, with proper capital

<a id="generate"></a>
# Step 5: Generate output using the foundation models Python library

You can prompt foundation models in watsonx.ai programmatically using the Python library.

See:
- <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-python-lib.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">Introduction to the foundation models Python library</a>
- <a href="https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html" target="_blank" rel="noopener no referrer">Foundation models Python library reference</a>


In [89]:
import os
from ibm_watson_machine_learning.foundation_models import Model

model_id = "ibm/granite-13b-chat-v2"

gen_parms = { 
    "DECODING_METHOD" : "greedy", 
    "MIN_NEW_TOKENS" : 1, 
    "MAX_NEW_TOKENS" : 100 
}

#project_id = os.environ["PROJECT_ID"]
project_id ="6353a079-5b03-4f44-bae4-04bab3ff2c9d"

model = Model( model_id, credentials, gen_parms, project_id )

In [90]:
import json

def generate( model_in, augmented_prompt_in ):
    
    generated_response = model_in.generate( augmented_prompt_in )

    if ( "results" in generated_response ) \
       and ( len( generated_response["results"] ) > 0 ) \
       and ( "generated_text" in generated_response["results"][0] ):
        return generated_response["results"][0]["generated_text"]
    else:
        print( "The model failed to generate an answer" )
        print( "\nDebug info:\n" + json.dumps( generated_response, indent=3 ) )
        return ""

In [91]:
output = generate( model, augmented_prompt )
print( output )


I don't know. The article does not provide specific information about black spot on jackfruit leaves.


<a id="rag"></a>
# Step 6: Pull everything together to perform retrieval-augmented generation

In [92]:
def searchAndAnswer(knowledge_base_in, model):
    question = input("Type your question:\n")
    if not re.match(r"\S+", question):
        print("No question")
        return

    # Retrieve the relevant content
    top_matching_index = search(question, knowledge_base_in)
    if top_matching_index < 0:
        print("No good answer was found in the knowledge base")
        # Now let's generate an answer using the model's knowledge
        model_answer = generate(model, question)
        print("Model's additional answer (based on its knowledge):\n" + model_answer)
        return

    asset = knowledge_base_in[top_matching_index]
    asset_txt = asset["txt"]

    # Augment a prompt with context
    augmented_prompt = augment(prompt_template, asset_txt, question)

    # Generate output
    output = generate(model, augmented_prompt)
    if not re.match(r"\S+", output):
        print("The model failed to generate an answer")
    print("\nAnswer:\n" + output)
    print("\nSource: \"" + asset["title"] + "\", " + asset["Author"])


Test the solution by running the following cell multiple times.  

\*You will be prompted to enter a question each time.

In [95]:
searchAndAnswer( knowledge_base, model )

Type your question:
coffee disease
No good answer was found in the knowledge base
Model's additional answer (based on its knowledge):
.

The disease is caused by the fungus Histoplasma capsulatum, which is found in soil contaminated by bird or bat droppings. It can grow in areas with high humidity and temperatures between 60 and 80 degrees Fahrenheit.

Symptoms of histoplasmosis typically appear between two to nine weeks after inhalation of the fungus. They include fever, headache, muscle aches, cough, chest pain, and sometimes diarrhea or nausea. In rare cases, it can cause more severe


In [52]:
import requests

url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"

body = {
"input": """You are an expert in diagnosing banana plant diseases. A user will describe symptoms of their banana plant, and you will provide a diagnosis and advice. Always confirm the symptoms by asking additional questions to improve accuracy.


Input: My banana leaves are turning yellow. What should I do?
Output: Could you describe any additional symptoms? Are there wilting leaves, black streaks, or any changes in fruit quality?


Input: black streaks on banana leaves

Output: Black Sigatoka in banana plants 


Input: pink patches on jackfruit branches
Output: Pink Disease in jackfruit

Input: My banana leaf turning yellow
Output:""",
    "parameters": {
        "decoding_method": "greedy",
        "max_new_tokens": 100,
        "min_new_tokens": 10,
        "stop_sequences": ["\n\n"],
        "repetition_penalty": 2
    },
    "model_id": "ibm/granite-13b-chat-v2",
    "project_id": "6353a079-5b03-4f44-bae4-04bab3ff2c9d",
    "moderations": {
        "hap": {
            "input": {
                "enabled": True,
                "threshold": 0.5,
                "mask": {
                    "remove_entity_value": True
                }
            },
            "output": {
                "enabled": True,
                "threshold": 0.5,
                "mask": {
                    "remove_entity_value": True
                }
            }
        }
    }
}

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    #"Authorization": "Bearer YOUR_ACCESS_TOKEN"
    "Authorization":cloud_apikey
}

response = requests.post(
    url,
    headers=headers,
    json=body
)

if response.status_code != 200:
    raise Exception("Non-200 response: " + str(response.text))

data = response.json()

Exception: Non-200 response: {"errors":[{"code":"authentication_no_token","message":"Failed to authenticate the request due to no Bearer token in the request header","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"b50803d5ca2a4d133db95668192986bc","status_code":401}

In [36]:
!curl -H "Authorization: Bearer {cloud_apikey}" -X {post} "https://us-south.ml.cloud.ibm.com/{ml/v1/text/generation?version=2023-05-29}"


<a id="summary"></a>
# Summary and next steps

You successfully completed this notebook!.
 
You learned how to apply the general retrieval-augmented generation pattern with a simple search component and a small knowledge base using watonx.ai.
 
Check out our <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">Documentation</a> for more samples, tutorials, documentation, and how-tos. 

<a id="cpd-only"></a>
# Other versions of this notebook

If you are using an on-premises version of IBM watsonx.ai, then use one of the following notebooks:
- <a href="https://github.com/IBMDataScience/sample-notebooks/blob/master/CloudPakForData/notebooks/4.8/Introduction-to-retrieval-augmented-generation.ipynb" target="_blank" rel="noopener no referrer">watsonx.ai version 1.0 on-premises
- <a href="https://github.com/IBMDataScience/sample-notebooks/blob/master/CloudPakForData/notebooks/5.0/Introduction-to-retrieval-augmented-generation.ipynb" target="_blank" rel="noopener no referrer">watsonx.ai version 2.0 on-premises


### Authors

**Sarah Packowski**, AI ContentOps - IBM Data and AI.

Copyright © 2023-2024 IBM. This notebook and its source code are released under the terms of the MIT License.